Install necessary libraries

In [1]:
!pip -q install langchain openai tiktoken pypdf faiss-cpu langchainhub
!pip install -U langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.9/815.9 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 66.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.2/241.2 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

Declare OPENAI API KEY

In [2]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [3]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_openai import ChatOpenAI,OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.runnables import RunnableParallel
from langchain_core.runnables import RunnableLambda
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain.schema import format_document
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

In [4]:
PATH = "/content/1.pdf" #your PDF path

loader = PyPDFLoader(PATH)
pages = loader.load_and_split()

#pages will contain metadata like page and source pdf as well

Now split the full text into chunks

In [5]:
# Splitting up the text into smaller chunks for indexing
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200, #striding over the text
    length_function = len,
)


In [6]:
docs_new = text_splitter.split_documents(pages)


In [7]:
len(docs_new)

49

In [8]:
docs_new[17].page_content

'ChengduEbyteElectronicTechnologyCo,;Ltd E32-433T30Dusermanual\nCopyright©2012–2019，ChengduEbyteElectronicTechnologyCo.,Ltd. 8\n5Functiondescription\n5.1Fixedtransmission\n5.2Broadcastingtransmission\n5.3Broadcastingaddress\n\uf06cForexample:SettheaddressofmoduleAas0xFFFF,andthechannelas0x04;\n\uf06cWhenmoduleisthetransmitter(transparenttransmission),allmodulesunderchannel0x04willreceivethedata,the'

In [9]:
docs_new[17].metadata

{'source': '/content/1.pdf', 'page': 8}

Now Embeddings of the data

In [10]:
embeddings = OpenAIEmbeddings()

In [11]:
db = FAISS.from_documents(docs_new, embeddings)

Retriever

In [12]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [60]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7da214bc7040>, search_kwargs={'k': 6})

RAG using agent

In [13]:

from langchain.tools.retriever import create_retriever_tool

tool = create_retriever_tool(
    retriever,
    "search_about_E32_from_available_document",
    "Searches and returns excerpts about E32 mentioned in the document given."
)

tools = [tool]

In [14]:
tools

[Tool(name='search_about_E32_from_available_document', description='Searches and returns excerpts about E32 mentioned in the document given.', args_schema=<class 'langchain.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x7d2e5c06d240>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d2e5c478ac0>, search_kwargs={'k': 6}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x7d2e5c06d360>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x7d2e5c478ac0>, search_kwargs={'k': 6}), document_prompt=PromptTemplate(input_variables=['page_content'], template='{page_content}'), document_separator='\n\n'))]

In [15]:


from langchain import hub

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [16]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0)

In [17]:


from langchain.agents import AgentExecutor, create_openai_tools_agent

agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools =  tools)



In [18]:
result = agent_executor.invoke({"input": "what is E32 in mentioned document?"})

In [19]:
result["output"]

'E32 is mentioned in the document "E32-433T30D User Manual" from Chengdu Ebyte Electronic Technology Co., Ltd. It appears to be a wireless module with different variations such as E32-915T30DSX1276915M, E32-170T30DSX1278170M, E32-868T20DSX1276868M, E32-915T20DSX1276915M, E32-433T20DCSX1278433M, and E32-433T30DSX1278433M. The document provides information about the specifications, features, and limitations of these modules.'

In [20]:
result = agent_executor.invoke({"input": "who manufacture E32 and what are some specifications of it?"})

In [70]:
result["output"]

"The E32 is manufactured by Chengdu Ebyte Electronic Technology Co., Ltd. Unfortunately, I couldn't find detailed specifications for the E32 module in the available documents."

In [21]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

In [22]:
message_history = ChatMessageHistory()

In [23]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: message_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [26]:
agent_with_chat_history.invoke(
    {"input": "what is E32 in mentioned document?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': 'what is E32 in mentioned document?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='Your name is Bob.')],
 'output': 'In the mentioned document, there are references to E32-915T30DSX1276915M, E32-170T30DSX1278170M, E32-868T20DSX1276868M, E32-915T20DSX1276915M, E32-433T20DCSX1278433M, and E32-433T30DSX1278433M. These are different models or versions of E32 wireless modules. The document also includes a user manual for the E32-433T30D module, which provides information about its specifications, features, and usage instructions.'}

In [28]:
agent_with_chat_history.invoke(
    {"input": "who manufacture it?"},
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    config={"configurable": {"session_id": "<foo>"}},
)

{'input': 'who manufacture it?',
 'chat_history': [HumanMessage(content="hi! I'm bob"),
  AIMessage(content='Hello Bob! How can I assist you today?'),
  HumanMessage(content="what's my name?"),
  AIMessage(content='Your name is Bob.'),
  HumanMessage(content='what is E32 in mentioned document?'),
  AIMessage(content='In the mentioned document, there are references to E32-915T30DSX1276915M, E32-170T30DSX1278170M, E32-868T20DSX1276868M, E32-915T20DSX1276915M, E32-433T20DCSX1278433M, and E32-433T30DSX1278433M. These are different models or versions of E32 wireless modules. The document also includes a user manual for the E32-433T30D module, which provides information about its specifications, features, and usage instructions.'),
  HumanMessage(content='who manufacture it?'),
  AIMessage(content='The manufacturer of the E32 wireless modules is Ebyte.')],
 'output': 'The manufacturer of the E32 wireless modules is Chengdu Ebyte Electronic Technology Co., Ltd.'}